# ECE 447: Multi-input / Multi-output

Eric Klavins|

Copyright &copy; University of Washington, 2019

# Code

In [228]:
import numpy as np
import scipy.integrate as spi
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from sympy import *
from matplotlib import animation

# Uncomment on Google colab
# !pip install JSAnimation
# !pip install control

from JSAnimation.IPython_display import display_animation
from control import * 

%matplotlib inline

# Comment out in Google colab 
init_printing(use_latex='mathjax')

# Uncomment in below Google colab to render sympy equations nicely
# def custom_latex_printer(exp,**options):
#     from google.colab.output._publish import javascript
#     url = "https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.3/latest.js?config=default"
#     javascript(url=url)
#     return printing.latex(exp,**options)

# MIMO for Transfer Functions

In MIMO systems, transfer functions are arranged into matrices. You can't avoid linear algebra!

Example
---

As an example, suppose we have the following system

<img width=75% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/mimo-tf.png" style="transform: rotate(2deg)">

By following the wires, we can write down relationships betweens the inputs, outputs, and blocks. For the first output:

\begin{align}
Y_1 & = G_1 G_2 U_1 + G_4 ( G_1 U_1 + G_3 U_2 ) \\
    & = ( G_1 G_2 + G_1 G_4 ) U_1 + G_3 G_4 U_2
\end{align}

And for the second output:

\begin{align}
Y_2 & = G_4 ( G_1 U_1 + G_3 U_2 ) \\
    & = G_1 G_4 U_1 + G_3 G_4 U_2 
\end{align}

Systems of linear equations are always more convenient in matrix form:

\begin{align}
{\bf Y}(s) & = \begin{pmatrix}
G_1(s) G_2(s) + G_1(s) G_4(s) & G_3(s) G_4(s) \\
G_1(s) G_4(s) & G_3(s) G_4(s) 
\end{pmatrix} {\bf U}(s) \\
 & = {\bf T}(s) {\bf U}(s)
\end{align}

The only thing you can't do is write $Y(s) / U(s)$ for vectors of transfer functions. 

Composition
---

Compositions go through as usual, as long as the size of the matrices are compatible.

**Series:**

<img width=65% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/mimo-series.png">

- $G_1$ is $m \times n$
- $G_2$ is $p \times m$
- $G_1 G_2$ is $p \times n$

Note that the composition $G_1 G_2$ is just matrix multiplication.

**Feedback:**

<img width=55% src="https://raw.githubusercontent.com/klavins/ECE447/master/images/mimo-fb.png">

In the feedback case, we have

$$
Y = GE = G(U-Y) = GU-GY
$$

Therefore, 

\begin{align}
Y+GY & = GU \\
(I+G)Y & = GU \\ 
Y & = (I+G)^{-1}GU
\end{align}

Assuming $I+G$ is invertible.

# MIMO in State Space

# Observer with Full State Feedback

The observer we designed in the previous notebook takes $y$ and $u$ as inputs and outputs $\hat{\x}$. Thus it is a two input, $n$ output system. 